# Intro
This code is used to make pull requests from inaturalist with authorization. It takes a list of taxonIds makes a dictionary out of it with taxonId as the key and the scientific name as the value. Then it gets the authorization via getaccesstoken to allow for pulling of all data rather than being capped. These two functions feed their data into dataPuller to iterate through the dictionary and use the authorization to pull each species data and store it in a list before writing it out to a taxonID titled csv with headers of id,location,observed_on, and time_observed_at as this is the only data we are interested in.

The Code Works with the functions: 
* listOfIdNames 
* getAccessToken
* dataPuller

imports:
* requests for http requests
* json to parse jason return
* csv to write csv files
* subprocess to allow clearing of screen during runs



In [17]:
import requests

In [2]:
import json

In [3]:
import csv

In [4]:
import subprocess as sp

## listOfIdsNames
This function opens up the file of your choosing with two columns in a text file, the first will be the key, the second its value. It's hardcoded but later may include it as a parameter instead. Once it is done iterating through the file it returns the dictionary.

In [7]:
def listOfIdsNames():
    file = open("taxon-Ids.txt").readlines()
    dictionary = {}
    count = 0
    for data in file:
        data = data.strip().split()
        if(count != 0):
            dictionary[data[0]] = data[1]+" "+data[2]
        else:       
            count = 1;
    return dictionary

## getAccessToken
This is the key needed to allow for uncapped data pulls.
* app_id and app_secret are the 24 hour tokens generated by the user on inaturalist
* username and password of an inaturalist user
* payload - is the dictionary of all the above values to allow for pass in for response
* response -  uses requests to get post value from the site with payload values to get the Bearer token
* headers - the actual authorization that is returned to be used in the dataPuller function

In [8]:
def getAccessToken():
    
    site = "https://www.inaturalist.org"
    app_id = '69345738565c2bd88f2dafa49857e426ad01918d5e5a72fcdde40d258f22b49c'
    app_secret = '62899ac1d355f1743b84db1e21e94f2bc40de4915cb7a2cb2afaeab41dfb0de8'
    username = 'ornelaseduardo'
    password = 'qb7A1PAl4eRp6rPh'

    payload = {
        'client_id': app_id,
        'client_secret': app_secret,
        'grant_type': "password",
        'username': username,
        'password': password
    }

    response = requests.post(("%s/oauth/token" % site), payload)
    token = response.json()["access_token"]
    headers = {"Authorization": "Bearer %s" % token}

    return headers

## dataPuller
This function takes a dictionary and authorization string and then begains to make get requests from inaturalists servers. There is a loading screen set up to let the user know the code is running and where it is currently at. The first get request provides how many pages are available for the requested taxonId. The pages are stored so a for loop can run that many times to make the request for each page, then runs a loop for each entry and saves the data in a list.Once all pages are completed it writes it out to a csv with the taxonId as the filename. id,location,observed_on, and time_observed_at are keys within the data that we pull from but you are able to get more data by simply adding another line with a valid key and appending it to the list. Ensure if you do so the header for the csv is updated with this new key value. 

### parameters and variables

* species - a dictionary passed it that is set up with the taxonID as the key and the scientific name as the value
* headers - the authorization string required to get uncapped data requests

* obs_data - note that we use a get requests using the inaturlist url and specify we want observations for taxonIDs that is then              concatinated in and that we also only accept research grade observations at this time
* jData - converts the obs_data into a json format (like a dictionary) so that we can iterate through
* templist - is a list of lists of all data from a species. This format is used as csv writer needs to iterate through data.                  Once a species taxonID is complete and written to a csv the list empties for the next species.
* holder - this is a list that holds data for one observation and is then appended to the templist 


** The if statments that creates our page amount is needed in the event that there are observations counts that are say 32 observations. If this is the case there are two pages that need to be pulled from where as if there are 29 observations only one page needs to be pulled from





In [10]:
def dataPuller(species,headers):
    print("Running...")

    run = 1
    for butterfly in species:
        
        percent = round((run/len(species))*100,1)
        
        print(str(percent)+"%")
       

        obs_data = requests.get(("http://api.inaturalist.org/v1/observations?taxon_id=" + str(butterfly) +"&quality_grade=research&page=1"), headers=headers)
        jData = json.loads(obs_data.text)
        total_Observations = int(jData["total_results"])
        pages=0
        if( total_Observations % 30 != 0):
            pages = (total_Observations//30)+1
        else:
            pages = total_Observations//30

        run2 = 1
        tempList = []
        for i in range(1,pages):
            
            percent2 = round((run2/pages)*100,1)
            print(str(percent)+"%")
            print("... "+str(run2)+" of "+str(pages)+" pages")

            obs_data = requests.get(("http://api.inaturalist.org/v1/observations?taxon_id=" + str(butterfly) +"&quality_grade=research&page="+str(i)), headers=headers) #TODO ids

            data = json.loads(obs_data.text)
            
            for records in data['results']:

                holder = []

                holder.append(records["id"])
                holder.append(str(butterfly))
                holder.append(str(species[butterfly]))
                
                if(records["location"] is not None):
                    latlong = records["location"].split(",")

                    holder.append(latlong[0])
                    holder.append(latlong[1])
                holder.append(records["observed_on"])
                holder.append(records["time_observed_at"])
                tempList.append(holder)

            run2+=1

            sp.call('cls',shell=True)
        run+=1

        with open(str(butterfly)+".csv", "w",encoding='utf-8') as file:
            writer = csv.writer(file)

            writer.writerows(tempList)
        tempList = []

In [13]:
def main():
    print("Running")
    butterflys = listOfIdsNames()
    key = getAccessToken()
    dataPuller(butterflys,key)
    print("Complete")

main()